In [13]:
#Imports

#Data
import pandas as pd
import numpy as np

#Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

#ML
from imblearn.under_sampling import RandomUnderSampler

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler


from sklearn.metrics import mean_squared_error, mean_absolute_error

from sklearn.linear_model import LinearRegression
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
import tensorflow as tf
from sklearn.neural_network import MLPRegressor
from sklearn import datasets
from sklearn.datasets import make_regression
from sklearn.metrics import confusion_matrix,classification_report

from sklearn.preprocessing import OneHotEncoder

#-------------------------------------------------------------------

data_file='wrang_xyz_data.csv'

financial_data=['total_rev_hi_lim','tot_cur_bal','tot_coll_amt','revol_util','loan_amnt','dti','revol_bal']
loan_data=['purpose','initial_list_status','term']
emp_data=['emp_length','collections_12_mths_ex_med','acc_now_delinq','home_ownership','annual_inc','verification_status','zip_code','addr_state','delinq_2yrs','inq_last_6mths','open_acc','pub_rec','total_acc']
date_data=['earliest_cr_line']
pred_data=['issue_d','default_ind','recoveries','collection_recovery_fee']
inn=['initial_list_status','last_pymnt_amnt','total_pymnt','total_rec_int','total_rec_late_fee']
out=['initial_list_status','last_credit_pull_d','out_prncp','last_pymnt_amnt','total_pymnt','total_rec_int','total_rec_late_fee']
features=financial_data+loan_data+emp_data+date_data+inn+out

In [14]:
def get_model_class(data_file='wrang_xyz_data.csv',split='date',model_type='xgb',pred_value=['recoveries']):

    df = pd.read_csv('data/'+data_file,usecols=features+['issue_d','default_ind']+pred_value)
    df=df[df['default_ind']==1].drop('default_ind',axis=1)
    if split=='random':
        X=df.drop([pred_value[0],'issue_d'],axis=1)
        y=df[pred_value[0]]

        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, test_size=0.25)

    else:
        X=df.drop(pred_value[0],axis=1)
        y=df[[pred_value[0],'issue_d']]

        X_train=X[X['issue_d']<=20150501].drop('issue_d',axis=1)
        X_test=X[X['issue_d']>20150501].drop('issue_d',axis=1)

        y_train=y[y['issue_d']<=20150501][pred_value[0]]
        y_test=y[y['issue_d']>20150501][pred_value[0]]

    if model_type=='xgb':
        model_reg = xgb.XGBRegressor()
        model_reg.fit(X_train,np.ravel(y_train))

    elif model_type=='lin_reg':
        model_reg=LinearRegression()
        model_reg.fit(X_train,np.ravel(y_train))

    else: 
        model_reg = RandomForestRegressor()
        model_reg.fit(X_train,np.ravel(y_train))

    y_reg_pred = model_reg.predict(X_test)
    error = np.mean(abs(y_test - y_reg_pred)/y_reg_pred)*100
    print(error)
    return model_reg
    
model_reg=get_model_class(split='date',model_type='xgb',pred_value=['recoveries'])

C:\Users\mmoy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


32.84670458467195
